<a href="https://colab.research.google.com/github/ManuSinghYadav/Gemma-7b-ft-QLoRA-300-Alpaca/blob/main/inferencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Notebook](https://colab.research.google.com/drive/1w7qXUTXBvUMnr1FKPs0mGn9HTp-YFlHW?authuser=1#scrollTo=khahtLltdRzL) used for finetuning

##Importing libraries & Logging in

In [1]:
!pip install -q peft datasets evaluate trl bitsandbytes transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━

In [31]:
import torch
import pandas as pd
import transformers
import accelerate
import huggingface_hub
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

In [3]:
huggingface_hub.login(token = userdata.get('HF_TOKEN_WRITE'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


##Downloading Base Model and Tokenizer

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
base_model_id = "google/gemma-7b"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True,
    offload_folder="offload/",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

##Merging base model and finetuned LoRA adapters

In [6]:
ft_model = PeftModel.from_pretrained(base_model, "msinghy/gemma-7b-ft-80row-alpaca-correcting-mistakes", offload_folder="offload/")

adapter_config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/100M [00:00<?, ?B/s]

##Inferencing

In [21]:
t = "What happens when you bite your nails?"
text = "###Input: " + t + "\n\n###Output: "

device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

output_ft_model = ft_model.generate(**inputs, max_new_tokens=500)

print("\n-----------------------\nFT Model's output (merged): \n-----------------------")
print(tokenizer.decode(output_ft_model[0], skip_special_tokens=True))


-----------------------
FT Model's output (merged): 
-----------------------
###Input: What happens when you bite your nails?

###Output: 1. Nail biting can lead to poor oral hygiene: When you bite your nails, you are introducing bacteria and other germs from your hands into your mouth, which can increase the risk of cavities, gum disease, and other oral health problems.

2. It can cause damage to the nails: Biting your nails can cause them to become brittle, thin, and weak, and can also lead to hangnails, cuticles, and other nail damage.

3. It can affect self-esteem: Nail biting can be a habit that is difficult to break, and can lead to feelings of embarrassment and low self-esteem.

4. It can be a sign of stress or anxiety: Nail biting is often a coping mechanism for stress or anxiety, and can be a sign of underlying emotional issues.

5. It can be a health hazard: In severe cases, nail biting can lead to skin infections, bleeding, and even tissue damage.
It's important to try to b

You can download the base model and run inference on it. You'll receive a completely different output. However, remember not to merge it with LoRA adapters. If you do so, it will become a merged model and produce the same output.